# 6 УРОК ETL (PYTHON + SQL)

## Как подключиться к ClickHouse через pandahouse?
1. Импортируем библиотеку:

In [1]:
#pip install pandahouse  # если библиотека ещё не установлена
import pandahouse as ph
import pandas as pd

##### 2. Далее создаем словарь connection с нужными параметрами:

для подключения к default:

In [2]:
connection_default = {'host': 'http://clickhouse.beslan.pro:8080',
                      'database':'default',
                      'user':'student', 
                      'password':'dpo_python_2020'
                     }

для подключения к test, где есть права на создание табличек и их модификацию:

In [3]:
connection_test = dict(database='test',
                  host='http://clickhouse.beslan.pro:8080',
                  user='student-rw',
                  password='656e2b0c9c')

##### Примеры
1. Пример: выводим первые 10 UserID, записываем в датафрейм q_test

In [4]:
# создаем запрос, db автоматически заменится на значение из database
q = '''
    SELECT 
        UserID 
    FROM 
        {db}.checks 
    LIMIT 10
    '''

# отправляем запрос и записываем результат в пандасовский датафрейм
q_test = ph.read_clickhouse(query=q, connection=connection_default)
q_test

,UserID
0,18446583642950580515
1,18446535622689003675
2,18446130411954852964
3,18446003252714243011
4,18445948434655311802
5,18445927732647659917
6,18445761122620052505
7,18445655133428855896
8,18445587876544434519
9,18445586451093345117


###### 2. Пример: создаем таблицу с помощью CREATE TABLE

In [5]:
q = """
CREATE TABLE test.example_table_s_kamsarin (
    id UInt32,
    supplementary_id UInt32,
    comments String 
) 
ENGINE = MergeTree 
ORDER BY (id, supplementary_id)
"""

ph.execute(query=q, connection=connection_test)

ClickhouseException: b'Code: 57. DB::Exception: Table test.example_table_s_kamsarin already exists. (TABLE_ALREADY_EXISTS) (version 21.12.2.17 (official build))\n'

###### 3. Пример записи данных в ClickHouse в уже созданную таблицу medium:

In [15]:
# записываем данные из pandas в clickhouse
ph.to_clickhouse(df, 'medium', index=False, connection=connection_test)

NameError: name 'example_table_s_kamsarin' is not defined

In [11]:
# записываем данные из pandas в clickhouse
ph.to_clickhouse(q_test, 'example_table_s_kamsarin', index=False, connection=connection_test)

ClickhouseException: b'Code: 16. DB::Exception: No such column UserID in table test.example_table_s_kamsarin (c53b13a3-cc19-4ced-853b-13a3cc190ced). (NO_SUCH_COLUMN_IN_TABLE) (version 21.12.2.17 (official build))\n'

In [12]:
# проверяем, что записалось
query_test = """
SELECT *
FROM test.example_table_s_kamsarin
LIMIT 100
"""

df_new = ph.read_clickhouse(query_test, connection=connection_test)
df_new

,id,supplementary_id,comments


In [16]:
# проверяем, что записалось
query_test = """
SELECT *
FROM test.medium
LIMIT 100
"""

df_new = ph.read_clickhouse(query_test, connection=connection_test)
df_new

ClickhouseException: b"Code: 60. DB::Exception: Table test.medium doesn't exist. (UNKNOWN_TABLE) (version 21.12.2.17 (official build))\n"